# Setup

In [1]:
 #! pip install python-twitter-v2

     |████████████████████████████████| 203 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 11.8 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
      Successfully uninstalled urllib3-1.26.7
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

selenium 4.0.0 requires urllib3[secure]~=1.26, but you'll have urllib3 1.25.11 which is incompatible.


In [ ]:
# ! git clone https://github.com/zacanger/profane-words

In [21]:
import pytwitter
from pytwitter import Api
import getpass
import pandas as pd
import re
import json
import flask
from flask import Flask, render_template_string, redirect, request

In [2]:
f = open("./profane-words/words.json", 'r')
bad_words = json.load(f)

# Init

In [3]:
bearer_token = getpass.getpass()
api = Api(bearer_token)

········


# Extract Timeline

In [4]:
def scrape_tl(user_id,count):
    out_text=[]
    temp_list=[]
    min_id = api.get_timelines(uid, max_results=5).data[0].id
    while len(out_text) < int(count):
        resp=api.get_timelines(uid, max_results=100, until_id = min_id)
        resp.data
        min_id = resp.data[-1].id
        for i in resp.data:
            out_text.append(i)
    out_text
    for item in out_text:
        temp_list.append([item.id,item.text])
    timeline_df = pd.DataFrame(temp_list, columns = ['id','text'])
    return timeline_df

# Regex language check

In [5]:
def flag_check(flag_list,text):
    flag_count = 0
    flags_found = []
    for flag in flag_list:
        if len(re.findall(f" {flag} ",text)) > 0:
            flag_count += 1
            flags_found.append(flag)
    return flag_count,flags_found

In [6]:
uid = 3241550339

In [7]:
df = scrape_tl(uid,200)

In [8]:
df['flags'] = df['text'].apply(lambda x : flag_check(bad_words,x))
df['occurance'] = df['flags'].apply(lambda x : 1 if x[0]>0 else 0)
df

,id,text,flags,occurance
0,1464703359806492680,RT @UMBCAthletics: ACC WIN FOR @UMBC_MBB AT PI...,"(0, [])",0
1,1462456040658817033,good morning from Virginia 🍁🌤 https://t.co/ctR...,"(0, [])",0
2,1462092327418269700,RT @gregorybrothers: @nbcwashington play this ...,"(0, [])",0
3,1461433533822611460,RT @JDDsays: Pat Collins is the best. Period.,"(0, [])",0
4,1458233458413744133,@nextChrist NICOLE!,"(0, [])",0
...,...,...,...,...
293,1047613127012110339,"@bentleyjcorbett who are you, me?","(0, [])",0
294,1047298189957709824,RT @NLEMuseum: Ride-along with us! Check out o...,"(0, [])",0
295,1046549036931698688,RT @axios: If Kavanaugh falls after the FBI in...,"(0, [])",0
296,1045417611301974016,RT @axios: Sen. Lindsey Graham to committee De...,"(0, [])",0


In [193]:
df[df.occurance==1].text.values[0]

'RT @owermohle: .@RepDianaDeGette says that if she and Rep. Tom Reed don\'t see a larger drug pricing bill "that we think is bipartisan," the…'

# Build Training Set

In [9]:
espn = 2557521
bs = 22637974
nba = 19923144
nfl = 19426551
hh = 20444254
sc = 26257166
lj = 23083404
tb = 1108050829393707008
pm = 353178677
kd = 35936474


accounts = [
    espn,
    bs,
    nba,
    nfl,
    hh,
    sc,
    lj,
    tb,
    pm,
    kd
]

In [13]:
user_list = []

for account in accounts:
    try:
        followers = api.get_followers(account).data
        for user in followers:
            user_list.append(user.id)
    except:
        print('f')


In [14]:
scrape_list = user_list

In [15]:
scrape_list

['1471560168252928001',
 '1471562433948962822',
 '1471562099461525504',
 '1471559757127376904',
 '1471562321579454465',
 '1471561625647951880',
 '1471561938605772801',
 '1413207584128049156',
 '1435134890794577920',
 '1421899241878917128',
 '1471557702228725762',
 '1470408659922341899',
 '1471561392532512768',
 '1471548248238092288',
 '1471562148509732865',
 '909061963',
 '1471561128811577344',
 '1471561000277184513',
 '1471561665565147147',
 '1471559060113735681',
 '1471559269912821773',
 '1471562057896112140',
 '1471559841135173635',
 '1471560004893397003',
 '1471561473403072521',
 '1471558789153411075',
 '1471561973703847944',
 '1471560525867855876',
 '1471561688339992580',
 '1471561520421109760',
 '1471561241248288772',
 '1471559259087273987',
 '1471558110363934720',
 '1471561647030476803',
 '1471561708657332228',
 '1471559215537963008',
 '1268377313042817029',
 '1471561607532531717',
 '1471559535357800450',
 '1471561640218869763',
 '1471560871977488386',
 '1471561679041437700',
 '

In [ ]:
dfs = []
for uid in scrape_list:
    try:
        temp_df = scrape_tl(uid,200)
        dfs.append(temp_df)
    except:
        print('fail')

fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail


In [ ]:
len(dfs)

In [47]:
train = pd.concat(dfs)

In [49]:
train['flags'] = train['text'].apply(lambda x : flag_check(bad_words,x))
train['occurance'] = train['flags'].apply(lambda x : 1 if x[0]>0 else 0)
train

,id,text,flags,occurance
0,1466258545297571854,@SamWebb77 I know sometimes he plays out of co...,"(0, [])",0
1,1466258079922757636,@trav2884 @SamWebb77 Wondering the same myself...,"(0, [])",0
2,1465769984324427777,"@TheEight8all In most cases, agree with you…no...","(0, [])",0
3,1465763568700346379,@TheEight8all @eightballmoves Just don’t….not ...,"(0, [])",0
4,1465538006010540034,@davebirkett @freep “Detroit Lions are ‘trying...,"(0, [])",0
...,...,...,...,...
195,1445451515649400841,@lherrero @tatitohdz @SrLebron por favor dale ...,"(0, [])",0
196,1445450664440569862,@SrLebron Esa es la fama que tiene en la NBA y...,"(0, [])",0
197,1445449897994375177,@SrLebron Michael Beasley no le queda como a K...,"(0, [])",0
198,1444401924065959938,Saludos a los Yankees! https://t.co/akKrlzks5b,"(0, [])",0


In [50]:
train['occurance'].value_counts(normalize = True)

0    0.968488
1    0.031512
Name: occurance, dtype: float64

## Launch Website

In [3]:
# Run in debug to get link
app = Flask(__name__) 

@app.route("/") 
def home(): 
    return render_template_string(HTML_PAGE)
app.run(debug = True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with fsevents reloader


SystemExit: 1

/Users/jackcummings/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [52]:
homePage = '''
<!doctype html>
<html>
    <head>
        <!-- Bootstrap CSS --> 
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
    </head>
    <body>
        <div class="container" style="padding:10px">
                <div class="card mb-3 h-100">
                    <div class="card-header">Welcome to socAIl! Please Enter your User ID Below </div>     
                    <div class="card-body"> 
                        <form method="POST" action="setUser"> 
                            <div class="form-group" role="group" > 
                                <label for="setUser">Please Enter Your User ID Here</label>
                                <input type="text" class="form-control" id="setUser" placeholder="@ESPN" name=user>
                            </div>
                                <button type="submit" class="btn btn-primary">Submit</button>
                            </div>
                        </form> 
                    </div>    
                </div>
        </div>
    </body>
</html>    '''

In [53]:
returnPage = '''
<!doctype html>
<html>
   <body>
     <strong>some text {} </strong>
   </body>
</html>
'''

In [54]:
app = Flask(__name__) 

@app.route("/", methods=['GET']) 
def home(): 
    return render_template_string(homePage)

@app.route("/setUser", methods=["POST"])
def setUser():
    user = request.form["user"]
    return render_template_string(returnPage.replace('{}',user))

app.run(debug = False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Dec/2021 20:45:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2021 20:45:26] "POST /setUser HTTP/1.1" 200 -
